<a href="https://colab.research.google.com/github/Shrey-Viradiya/PyTorch_for_DL/blob/master/PyTorch_Convolution_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch_Convolution Neural Network

In [1]:
!nvidia-smi

Fri Oct  2 05:45:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!rm -r data train test val

rm: cannot remove 'data': No such file or directory


In [3]:
! mkdir ~/.kaggle
! cp '/content/drive/My Drive/kaggle.json' ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d lucassj/dogs-vs-cats-train-validadion-and-evaluation
! unzip -n '/content/dogs-vs-cats-train-validadion-and-evaluation.zip'

Streaming output truncated to the last 5000 lines.
  inflating: validation/cat/cat.10001.jpg  
  inflating: validation/cat/cat.10002.jpg  
  inflating: validation/cat/cat.10003.jpg  
  inflating: validation/cat/cat.10004.jpg  
  inflating: validation/cat/cat.10005.jpg  
  inflating: validation/cat/cat.10006.jpg  
  inflating: validation/cat/cat.10007.jpg  
  inflating: validation/cat/cat.10008.jpg  
  inflating: validation/cat/cat.10009.jpg  
  inflating: validation/cat/cat.10010.jpg  
  inflating: validation/cat/cat.10011.jpg  
  inflating: validation/cat/cat.10012.jpg  
  inflating: validation/cat/cat.10013.jpg  
  inflating: validation/cat/cat.10014.jpg  
  inflating: validation/cat/cat.10015.jpg  
  inflating: validation/cat/cat.10016.jpg  
  inflating: validation/cat/cat.10017.jpg  
  inflating: validation/cat/cat.10018.jpg  
  inflating: validation/cat/cat.10019.jpg  
  inflating: validation/cat/cat.10020.jpg  
  inflating: validation/cat/cat.10021.jpg  
  inflating: validation/c

In [4]:
# !unzip -n '/content/drive/My Drive/images.zip'

## Import Libraries

In [5]:
import torch
import torchvision
from PIL import Image
import time

# CNNNet (or AlexNet by another name…)

In [6]:
class CNNNet(torch.nn.Module):

    def __init__(self, num_classes = 2):
        super(CNNNet, self).__init__()
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3,64, kernel_size=11, stride=4, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=3, stride = 2),
            torch.nn.Conv2d(64, 192, kernel_size = 5, padding = 2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(192, 384, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(384, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = torch.nn.AdaptiveAvgPool2d((6,6))
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(),
            torch.nn.Linear(256 * 6 * 6, 4096),
            torch.nn.ReLU(),
            torch.nn.Dropout(),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(),
            torch.nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [7]:
cnnnet = CNNNet()

In [8]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    
    for epoch in range(epochs):
        start = time.time()
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        correct = 0 
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs, targets = batch
                inputs = inputs.to(device)
                output = model(inputs)
                targets = targets.to(device)
                loss = loss_fn(output,targets) 
                valid_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        valid_loss /= len(val_loader.dataset)

        print(f'Epoch: {epoch}, Training Loss: {training_loss:.2f}, Validation Loss: {valid_loss:.2f}, accuracy = {(correct / total):.2f}, time: {time.time() - start}')

In [9]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

img_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64,64)),
    torchvision.transforms.ColorJitter(),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    ])

train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "./validation/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)
batch_size=10
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [10]:
cnnnet.to(device)
optimizer = torch.optim.Adam(cnnnet.parameters(), lr=0.00005)

In [11]:
train(cnnnet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=25, device=device)

Epoch: 0, Training Loss: 0.06, Validation Loss: 0.06, accuracy = 0.70, time: 125.05465865135193
Epoch: 1, Training Loss: 0.05, Validation Loss: 0.05, accuracy = 0.77, time: 122.81562161445618
Epoch: 2, Training Loss: 0.05, Validation Loss: 0.05, accuracy = 0.79, time: 121.94584250450134
Epoch: 3, Training Loss: 0.04, Validation Loss: 0.04, accuracy = 0.79, time: 121.91100025177002
Epoch: 4, Training Loss: 0.04, Validation Loss: 0.05, accuracy = 0.78, time: 121.85887789726257
Epoch: 5, Training Loss: 0.04, Validation Loss: 0.04, accuracy = 0.80, time: 121.60627222061157
Epoch: 6, Training Loss: 0.03, Validation Loss: 0.04, accuracy = 0.80, time: 122.5108962059021
Epoch: 7, Training Loss: 0.03, Validation Loss: 0.05, accuracy = 0.79, time: 120.41695475578308
Epoch: 8, Training Loss: 0.03, Validation Loss: 0.05, accuracy = 0.80, time: 122.41351103782654
Epoch: 9, Training Loss: 0.03, Validation Loss: 0.05, accuracy = 0.79, time: 122.75190949440002
Epoch: 10, Training Loss: 0.02, Validatio


## Downloading a pretrained network
There are two ways of downloading pre-trained image models with PyTorch. Firstly, you can use the torchvision.models library, or you can use PyTorch Hub. The latter is preferred as of 2019, as this is a one-stop shop for all models and the new standard for distributing models with PyTorch.

In [12]:
import torchvision.models as models
alexnet = models.alexnet(num_classes=1000, pretrained=True)

In [13]:
resnet50 = torch.hub.load('pytorch/vision', 'resnet50')

Using cache found in /root/.cache/torch/hub/pytorch_vision_master


In [14]:
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [15]:
print(resnet50)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
alexnet.classifier = torch.nn.Sequential(
            torch.nn.Dropout(),
            torch.nn.Linear(256 * 6 * 6, 4096),
            torch.nn.ReLU(),
            torch.nn.Dropout(),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(),
            torch.nn.Linear(4096, 2)
        )

In [17]:
alexnet.to(device)
optimizer = torch.optim.Adam(alexnet.parameters(), lr=0.00005)

In [18]:
train(alexnet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=25, device=device)

Epoch: 0, Training Loss: 0.04, Validation Loss: 0.04, accuracy = 0.84, time: 114.83827948570251
Epoch: 1, Training Loss: 0.04, Validation Loss: 0.04, accuracy = 0.84, time: 114.16915845870972
Epoch: 2, Training Loss: 0.03, Validation Loss: 0.03, accuracy = 0.85, time: 115.19968914985657
Epoch: 3, Training Loss: 0.03, Validation Loss: 0.03, accuracy = 0.85, time: 114.92985272407532
Epoch: 4, Training Loss: 0.02, Validation Loss: 0.04, accuracy = 0.85, time: 114.98709869384766
Epoch: 5, Training Loss: 0.02, Validation Loss: 0.04, accuracy = 0.85, time: 114.97286581993103
Epoch: 6, Training Loss: 0.02, Validation Loss: 0.04, accuracy = 0.86, time: 115.00862884521484
Epoch: 7, Training Loss: 0.01, Validation Loss: 0.04, accuracy = 0.85, time: 114.64329528808594
Epoch: 8, Training Loss: 0.01, Validation Loss: 0.05, accuracy = 0.86, time: 114.20113921165466
Epoch: 9, Training Loss: 0.01, Validation Loss: 0.04, accuracy = 0.85, time: 114.54360580444336
Epoch: 10, Training Loss: 0.01, Validati